In [6]:
import os
def get_api_key(api_key_name):
    return os.getenv(api_key_name)

In [14]:
from typing import Annotated, TypedDict, Literal, List
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import requests
import tqdm as notebook_tqdm
from langchain.utilities import SerpAPIWrapper

In [8]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

## Example 01

In [11]:
WEATHER_API_KEY = get_api_key("WEATHER_API_KEY")
TAVILY_API_KEY = get_api_key("TAVILY_API_KEY")
GOOGLE_API_KEY = get_api_key("GOOGLE_API_KEY")

In [10]:
@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather"""
    endpoint = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={query}"
    response = requests.get(endpoint)
    data = response.json()

    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

@tool
def search_web(query: str) -> list:
    """Search the web for a query"""
    tavily_search = TavilySearchResults(api_key=TAVILY_API_KEY, max_results=2, search_depth='advanced', max_tokens=1000)
    results = tavily_search.invoke(query)
    return results

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, google_api_key=GOOGLE_API_KEY)

In [13]:
tools = [search_web, get_weather]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
#https://medium.com/pythoneers/building-ai-agent-systems-with-langgraph-9d85537a6326
#https://aiproduct.engineer/tutorials/langgraph-tutorial-mastering-toolnode-implementation-unit-22-exercise-6